In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import sqlalchemy
from sqlalchemy import create_engine, Table, MetaData, select, and_, desc, func, case, cast, Float
from sqlalchemy import Column, String, Integer, Boolean, insert, update, delete

import pymysql
import psycopg2
import pyodbc

import json

from pprint import pprint as pp
import csv

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.max_colwidth = None
pd.set_option("display.float_format", lambda x: '%.2f' % x)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
sqlalchemy.__version__

'2.0.6'

In [3]:
pyodbc.version

'4.0.35'

## Getting Credentials

In [4]:
test_postgres_file_path = "credentials/demo_test_postgres.json"
hackerman_file_path = "credentials/test_postgressql.json"

In [5]:
with open(test_postgres_file_path) as f:
    json_file = json.load(f)

In [6]:
def load_json(file_path):
    """ Loads json files """
    try:
        with open(file_path) as f:
            json_file = json.load(f)
            return json_file
    except Exception as e:
        print(str(e))

In [7]:
new_postgres_credential = load_json(test_postgres_file_path)
db_credentials = load_json(hackerman_file_path)

### Data Ingestion

In [8]:
census_data = 'data/census.csv'

In [9]:
census_sql_data = 'data/census.sqlite'

In [10]:
employee_sql_data = 'data/employees.sqlite'

## SQLite

In [11]:
from sqlalchemy import create_engine

In [12]:
engine = create_engine('sqlite:///data/census.sqlite')
connection = engine.connect()

## PostgresSQL

In [13]:
import psycopg2

In [14]:
# Connect to an existing database
# conn = psycopg2.connect("dbname=test user=postgres")

In [15]:
datahackerman_connection = psycopg2.connect(
    dbname= db_credentials["database"],
    user= db_credentials["username"],
    password=db_credentials["password"],
    host=db_credentials["host"],
    port=db_credentials["port"]

)

In [16]:
datahackerman_connection

<connection object at 0x000002B108AC3480; dsn: 'user=hackerman password=xxx dbname=free_hackerman host=freetier-hackerman.c9gajtac8ssy.eu-west-2.rds.amazonaws.com port=5432', closed: 0>

In [17]:
datahackerman_cur = datahackerman_connection.cursor()

In [18]:
datahackerman_cur.execute("SELECT * FROM Books LIMIT 5")

In [19]:
datahackerman_cur.fetchone()

(81490378501273,
 'Leslie Godwin',
 292,
 '1985-11-30',
 7,
 'Religious',
 'wispy Hiawatha Midwestern prolix sanitarium aloe')

In [20]:
# Make the changes to the database persistent
datahackerman_connection.commit()
# Close communication with the database
datahackerman_cur.close()
datahackerman_connection.close()


#### Getting data in Tabular form

In [21]:
def get_data_postgress_raw(query_string, db_credentials):
    connection = psycopg2.connect(
        dbname= db_credentials["database"],
        user= db_credentials["username"],
        password=db_credentials["password"],
        host=db_credentials["host"],
        port=db_credentials["port"]
    )
    
    cursor = connection.cursor()
    cursor.execute(query_string)
    
    #print(cursor.fetchone())
    
    result = cursor.fetchall()
    
    connection.commit()
    
    connection.close()
    cursor.close()
    
    return result

In [22]:
query_string = """
SELECT * FROM Books
"""

In [23]:
# get_data_postgress_raw(query_string, db_credentials)

In [24]:
book_data = get_data_postgress_raw(query_string, db_credentials)

In [25]:
len(book_data)

2999

In [26]:
book_data[0:10]

[(81490378501273,
  'Leslie Godwin',
  292,
  '1985-11-30',
  7,
  'Religious',
  'wispy Hiawatha Midwestern prolix sanitarium aloe'),
 (2265183237865,
  'Theresa Scruggs',
  51,
  '1922-02-15',
  30,
  'SciFi',
  'deterrent someday indent decennial'),
 (38355930389954,
  'Elaine Turner',
  51,
  '1990-02-27',
  30,
  'Religious Novels',
  'aquatic certificate flatus flowery slack'),
 (12676076159906,
  'Claretta Dewitt',
  308,
  '1988-06-23',
  7,
  'HISTORICAL',
  'pre scald pillage minor incomputable'),
 (17366175799537,
  'Douglas Barrio',
  243,
  '1961-01-29',
  30,
  'Fantasy',
  'hateful division Malagasy'),
 (94390875304793,
  'Donald Hoes',
  283,
  '1927-03-13',
  30,
  'pop science',
  'Shaffer deodorant torn armload giantess trident'),
 (59564182657149,
  'Sherri Campos',
  192,
  '1916-08-11',
  30,
  'SHORT-STORIES',
  'conserve aerate diatomic prognosticate melange'),
 (78141158410406,
  'Gerald Hundley',
  183,
  '1928-01-07',
  30,
  'SHORT STORIES',
  'Dolores zing 

In [27]:
pd.DataFrame(book_data[0:10])

,0,1,2,3,4,5,6
0,81490378501273,Leslie Godwin,292,1985-11-30,7,Religious,wispy Hiawatha Midwestern prolix sanitarium aloe
1,2265183237865,Theresa Scruggs,51,1922-02-15,30,SciFi,deterrent someday indent decennial
2,38355930389954,Elaine Turner,51,1990-02-27,30,Religious Novels,aquatic certificate flatus flowery slack
3,12676076159906,Claretta Dewitt,308,1988-06-23,7,HISTORICAL,pre scald pillage minor incomputable
4,17366175799537,Douglas Barrio,243,1961-01-29,30,Fantasy,hateful division Malagasy
5,94390875304793,Donald Hoes,283,1927-03-13,30,pop science,Shaffer deodorant torn armload giantess trident
6,59564182657149,Sherri Campos,192,1916-08-11,30,SHORT-STORIES,conserve aerate diatomic prognosticate melange
7,78141158410406,Gerald Hundley,183,1928-01-07,30,SHORT STORIES,Dolores zing Galveston Minoan davit
8,86413140521812,Sara Hammond,485,1955-03-01,7,mystery,inquiry official biotite ablution rodent Dan
9,30958141877211,Alexandra Sandoval,330,1919-07-15,30,Fantasy,Peggy Peruvian rhinoceros miscible speakeasy Wallis


#### Use ´pandas.read_sql´
###### https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html

In [28]:
datahackerman_connection = psycopg2.connect(
    dbname= db_credentials["database"],
    user= db_credentials["username"],
    password=db_credentials["password"],
    host=db_credentials["host"],
    port=db_credentials["port"]

)

In [29]:
query_string = """
SELECT * FROM rentals
"""

In [30]:
%%time
rentals_data = pd.read_sql(query_string, datahackerman_connection)

<timed exec>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


CPU times: total: 1.39 s
Wall time: 15.5 s


In [31]:
rentals_data.head()

,UID,book,borrower,out,returned
0,76609620900561,37305790289730,88833280729411,2017-06-29,2017-07-17
1,83114291920228,1253436612142,88833280729411,2009-11-29,2009-12-16
2,88537859878214,92083778372643,88833280729411,2015-05-15,2015-06-09
3,75410000220725,51049394513918,88833280729411,2014-09-15,2014-10-04
4,32824255130354,14360217402682,88833280729411,2012-06-12,2012-06-20


In [32]:
# Alternative method to the issue above using SQLAlchemy.

def get_data_postgress_tabular(query_string, db_credentials):
    connection = psycopg2.connect(
        dbname= db_credentials["database"],
        user= db_credentials["username"],
        password=db_credentials["password"],
        host=db_credentials["host"],
        port=db_credentials["port"]
    )
    
    
    
    result = pd.read_sql(query_string, connection)
    
    return result

In [33]:
query_string = """
SELECT * FROM borrowers
"""

In [34]:
borrowers_data = get_data_postgress_tabular(query_string, db_credentials)

C:\Users\all4u\AppData\Local\Temp\ipykernel_1124\1178520926.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query_string, connection)


In [35]:
borrowers_data.head()

,DoB,UID,address,ethnic_group,gender,join_date
0,1964-05-26,6407399394844,M1 2,D,M,1981-08-20
1,1935-10-19,76509601043881,M24 4,D,M,1973-10-14
2,1937-03-01,6792108939008,M33 5,D,M,1974-02-25
3,1964-06-18,10123457846743,M5 4,F,M,1973-01-07
4,1968-10-24,28422585028351,M40 7,D,M,1984-10-18


#### Combine Books, Borrowers and Rentals data

In [36]:
combined_query_string = """
select
    r."UID" as rental_uid,
    r.book as book_uid,
    r.borrower as borrower_uid,
    r."out" as borrowed_date,
    r.returned as borrower_returned_date,
    b.author as book_author,
    b."length" as book_length,
    b.publication_date as book_publication_date,
    b.rental_period as book_rental_date,
    b.subject as book_subject,
    b.title as book_title,
    b2."DoB" as borrower_date_of_birth,
    b2.address as borrower_address,
    b2.ethnic_group as borrower_ethnic_group,
    b2.gender as borrower_gender,
    b2.join_date as borrower_join_date
from rentals r
left join books b
    on r.book = b."UID"
left join borrowers b2
    on r.borrower = b2."UID"
"""

In [37]:
combined_data = get_data_postgress_tabular(combined_query_string, db_credentials)

C:\Users\all4u\AppData\Local\Temp\ipykernel_1124\1178520926.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query_string, connection)


In [38]:
combined_data.head()

,rental_uid,book_uid,borrower_uid,borrowed_date,borrower_returned_date,book_author,book_length,book_publication_date,book_rental_date,book_subject,book_title,borrower_date_of_birth,borrower_address,borrower_ethnic_group,borrower_gender,borrower_join_date
0,76609620900561,37305790289730,88833280729411,2017-06-29,2017-07-17,Robert Acuff,362,1981-07-09,30,Teen,dachshund default partition amputate,2001-03-14,M1 3,D,F,2008-04-19
1,83114291920228,1253436612142,88833280729411,2009-11-29,2009-12-16,Meredith Saunders,414,1978-11-09,30,teen,phonic auspices Gaussian madmen produce expedition,2001-03-14,M1 3,D,F,2008-04-19
2,88537859878214,92083778372643,88833280729411,2015-05-15,2015-06-09,Hazel Law,308,1985-11-25,30,Short Stories,citron Danny foot,2001-03-14,M1 3,D,F,2008-04-19
3,75410000220725,51049394513918,88833280729411,2014-09-15,2014-10-04,Michael Dealba,315,1968-04-25,30,horror,sapient vivacious guillotine bushland shoal insouciant,2001-03-14,M1 3,D,F,2008-04-19
4,32824255130354,14360217402682,88833280729411,2012-06-12,2012-06-20,Christopher Smith,345,1974-12-19,7,fantasy,flounder blight taffeta supplicant accede,2001-03-14,M1 3,D,F,2008-04-19


### MS SQL Server

In [39]:
import pyodbc

```python
# For Trusted Connection:

connection_string = (
    "Driver={SQL Server Native Client 11.0};"
            "Server=Your_Server_Name;"
            "Database=My_Database_Name;"
            "Trusted_Connection=yes;"
)
```

```python
# For Non-Trusted Connection:

connection_string = (
    "Driver={SQL Server Native Client 11.0};"
            "Server=Your_Server_Name;"
            "Database=My_Database_Name;"
            "UID=Your_User_ID;"
            "PWD=Your_Password;"
)
```

In [40]:
# connection = pyodbc.connect(connection_string)

In [41]:
connection_string = (
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=datahackerman.c9gajtac8ssy.eu-west-2.rds.amazonaws.com;"
    "Database=hackerman;"
    "UID=admin;"
    "PWD=Genesis5:24#;"
)

In [42]:
connection = pyodbc.connect(connection_string)

In [43]:
from sqlalchemy.engine import URL

connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

In [44]:
connection_url

mssql+pyodbc://?odbc_connect=Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3BServer%3Ddatahackerman.c9gajtac8ssy.eu-west-2.rds.amazonaws.com%3BDatabase%3Dhackerman%3BUID%3Dadmin%3BPWD%3DGenesis5%3A24%23%3B

In [45]:
engine = create_engine(connection_url)

In [46]:
engine

Engine(mssql+pyodbc://?odbc_connect=Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3BServer%3Ddatahackerman.c9gajtac8ssy.eu-west-2.rds.amazonaws.com%3BDatabase%3Dhackerman%3BUID%3Dadmin%3BPWD%3DGenesis5%3A24%23%3B)

```python
# Initilise the cursor
cursor = connection.cursor()

# Executing a SQL Query
cursor.execute(query_string)
```

In [47]:
db_credentials["username"]

'hackerman'

In [48]:
new_postgres_credential["database"]

'postgres'

In [49]:
new_postgres_credential["host"]

'mytestinginstance.c7smiokcpmcb.eu-north-1.rds.amazonaws.com'

In [50]:
# Creating a connection
# Connection to Dayo's database

hackerman_connection = psycopg2.connect(
        dbname=db_credentials["database"],
        user=db_credentials["username"],
        password=db_credentials["password"],
        host=db_credentials["host"],
        port=db_credentials["port"]
    )

In [51]:
# Connection to own database
new_postgres_connection = psycopg2.connect(
        dbname=new_postgres_credential["database"],
        user=new_postgres_credential["username"],
        password=new_postgres_credential["password"],
        host=new_postgres_credential["host"],
        port=new_postgres_credential["port"]
    )

In [52]:
## Checking the connection

In [53]:
hackerman_connection

<connection object at 0x000002B109D497B0; dsn: 'user=hackerman password=xxx dbname=free_hackerman host=freetier-hackerman.c9gajtac8ssy.eu-west-2.rds.amazonaws.com port=5432', closed: 0>

In [54]:
new_postgres_connection

<connection object at 0x000002B109D498C0; dsn: 'user=postgres1 password=xxx dbname=postgres host=mytestinginstance.c7smiokcpmcb.eu-north-1.rds.amazonaws.com port=5432', closed: 0>

In [55]:
hackerman_cur = hackerman_connection.cursor()

In [56]:
hackerman_cur.execute("SELECT * FROM Books LIMIT 5;")

In [57]:
hackerman_cur.fetchone()

(81490378501273,
 'Leslie Godwin',
 292,
 '1985-11-30',
 7,
 'Religious',
 'wispy Hiawatha Midwestern prolix sanitarium aloe')

In [58]:
from sqlalchemy.engine import URL

hackerman_url_object = URL.create(
    "postgresql+psycopg2",
    username=db_credentials["username"],
    password=db_credentials["password"],
    host=db_credentials["host"],
    database=db_credentials["database"],
    port=db_credentials["port"]
)

In [59]:
engine = create_engine(hackerman_url_object)

In [60]:
query_string = """
SELECT * FROM rentals
"""

In [61]:
combined_query_string = """
select
    r."UID" as rental_uid,
    r.book as book_uid,
    r.borrower as borrower_uid,
    r."out" as borrowed_date,
    r.returned as borrower_returned_date,
    b.author as book_author,
    b."length" as book_length,
    b.publication_date as book_publication_date,
    b.rental_period as book_rental_date,
    b.subject as book_subject,
    b.title as book_title,
    b2."DoB" as borrower_date_of_birth,
    b2.address as borrower_address,
    b2.ethnic_group as borrower_ethnic_group,
    b2.gender as borrower_gender,
    b2.join_date as borrower_join_date
from rentals r
left join books b
    on r.book = b."UID"
left join borrowers b2
    on r.borrower = b2."UID"
"""
from sqlalchemy.engine import URL
from sqlalchemy.sql import text

hackerman_url_object = URL.create(
    "postgresql+psycopg2",
    username=db_credentials["username"],
    password=db_credentials["password"],
    host=db_credentials["host"],
    database=db_credentials["database"],
    port=db_credentials["port"]
)

engine = create_engine(hackerman_url_object)

with engine.connect().execution_options(autocommit=True) as dh_connection:
    # df = pd.read_sql(f"""SELECT * FROM table_name WHERE condition""", con = conn)
    combined_data = pd.read_sql(text(combined_query_string), dh_connection)

In [62]:
combined_data.head(1)

,rental_uid,book_uid,borrower_uid,borrowed_date,borrower_returned_date,book_author,book_length,book_publication_date,book_rental_date,book_subject,book_title,borrower_date_of_birth,borrower_address,borrower_ethnic_group,borrower_gender,borrower_join_date
0,76609620900561,37305790289730,88833280729411,2017-06-29,2017-07-17,Robert Acuff,362,1981-07-09,30,Teen,dachshund default partition amputate,2001-03-14,M1 3,D,F,2008-04-19


## Writing Data to Database

In [63]:
from sqlalchemy.engine import URL
from sqlalchemy.sql import text

new_hackerman_url_object = URL.create(
    "postgresql+psycopg2",
    username=new_postgres_credential["username"],
    password=new_postgres_credential["password"],
    host=new_postgres_credential["host"],
    database=new_postgres_credential["database"],
    port=new_postgres_credential["port"]
)

engine = create_engine(new_hackerman_url_object)

combined_data.head(100).to_sql("rental_details_another", con=engine, schema="public", if_exists='replace', index=False, chunksize=1000)

100

In [64]:
combined_query_string = """
select * from rental_details_another
"""

with engine.connect().execution_options(autocommit=True) as dh_connection:
    new_combined_data = pd.read_sql(text(combined_query_string), dh_connection)

In [65]:
new_combined_data.head(10)

,rental_uid,book_uid,borrower_uid,borrowed_date,borrower_returned_date,book_author,book_length,book_publication_date,book_rental_date,book_subject,book_title,borrower_date_of_birth,borrower_address,borrower_ethnic_group,borrower_gender,borrower_join_date
0,76609620900561,37305790289730,88833280729411,2017-06-29,2017-07-17,Robert Acuff,362,1981-07-09,30,Teen,dachshund default partition amputate,2001-03-14,M1 3,D,F,2008-04-19
1,83114291920228,1253436612142,88833280729411,2009-11-29,2009-12-16,Meredith Saunders,414,1978-11-09,30,teen,phonic auspices Gaussian madmen produce expedition,2001-03-14,M1 3,D,F,2008-04-19
2,88537859878214,92083778372643,88833280729411,2015-05-15,2015-06-09,Hazel Law,308,1985-11-25,30,Short Stories,citron Danny foot,2001-03-14,M1 3,D,F,2008-04-19
3,75410000220725,51049394513918,88833280729411,2014-09-15,2014-10-04,Michael Dealba,315,1968-04-25,30,horror,sapient vivacious guillotine bushland shoal insouciant,2001-03-14,M1 3,D,F,2008-04-19
4,32824255130354,14360217402682,88833280729411,2012-06-12,2012-06-20,Christopher Smith,345,1974-12-19,7,fantasy,flounder blight taffeta supplicant accede,2001-03-14,M1 3,D,F,2008-04-19
5,27743244165755,8395905212896,88833280729411,2013-09-17,2013-09-24,Dean Davis,180,1965-01-07,7,POP SCIENCE,corrigible extroversion Grimm joggle,2001-03-14,M1 3,D,F,2008-04-19
6,74758358433492,55222900491105,88833280729411,2015-05-10,2015-06-07,Charles Coffin,449,1926-06-23,30,POP SCIENCE,criteria substantiate Pickett bullet savvy,2001-03-14,M1 3,D,F,2008-04-19
7,80552630917389,94850060106421,88833280729411,2014-07-23,2014-08-19,Donna Carroll,694,2013-01-07,30,teen,constituent upheaval Laramie,2001-03-14,M1 3,D,F,2008-04-19
8,59319216887299,74536758781546,88833280729411,2008-10-18,2008-11-11,Elliot Heston,51,1934-04-22,30,LANGUAGES,indomitable Texan pea gracious Del fluke,2001-03-14,M1 3,D,F,2008-04-19
9,10291516659073,44946570298082,88833280729411,2010-10-20,2010-10-27,Eric Kendall,327,1991-08-29,7,short stories,carnage occipital bunkmate Felicia landau,2001-03-14,M1 3,D,F,2008-04-19
